In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy

import sys
sys.path.append("../")
import os

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-01-31 13:50:30 __main__ INFO     torch.__version__='2.5.0+cu124', torch.version.cuda='12.4'
2025-01-31 13:50:30 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-01-31 13:50:30 __main__ INFO     transformers.__version__='4.48.1'


#### Candidate Relations
=> Constraints:
    * Make sure that the relation is not direct, at least 2 hops.

* Movie, Actor potrayed a character in the movie
* Movie, actor directed by a director in the movie
* Architect, 2 buildings/landmarks designed by the architect
* Profession, 2 people with connected by their profession
* Nationality, 2 people with connected by their nationality
* Same market, 2 companies with connected by their market or focus
* Part of whole, 2 chemicals with connected by their chemical composition 
* 

In [3]:
os.listdir("/home/local_arnab/Codes/00_MODEL/meta-llama")

['Llama-3.1-8B',
 'Llama-3.1-8B-Instruct',
 'Llama-2-7b-chat-hf',
 'Llama-3.2-3B-Instruct',
 'Llama-3.2-3B',
 'Llama-3.2-1B']

In [4]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer

# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "Qwen/Qwen2-7B"
model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.float16,
)

2025-01-31 13:50:30 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.41s/it]


2025-01-31 13:50:36 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/deepseek-ai/DeepSeek-R1-Distill-Llama-8B> | size: 15316.508 MB | dtype: torch.float16 | device: cuda:0


In [5]:
from src.probing.utils import (
    ProbingPrompt,
    ProbingLatents,
    prepare_probing_input,
    get_lm_generated_answer,
    check_if_answer_is_correct,
)

In [6]:
import json
probe_class = "atheletes/basketball"
probe_file = os.path.join(
    env_utils.DEFAULT_DATA_DIR,
    "probe",
    probe_class if probe_class.endswith(".json") else f"{probe_class}.json",
) 

probe_data = json.load(open(probe_file, "r"))

probe_data["class"]

'basketball'

In [7]:
import itertools
import random

nc2 = list(itertools.combinations(probe_data["entities"], 2))
random.shuffle(nc2)
len(nc2)

11935

In [8]:
nc2[:5]

[('Hakeem Olajuwon', 'Manu Ginobili'),
 ('Kobe Bryant', 'Zion Williamson'),
 ('Ben Wallace', 'Michael Jordan'),
 ('Hakeem Olajuwon', 'Chris Bosh'),
 ('Victor Oladipo', 'James Worthy')]

In [9]:

# prompt = "What is the most visited place in Paris? Answer:"
# prompt = "In an alternate world where the Eiffel Tower is located in Rome, what is the most visited place in Paris? Answer:"
# prompts = [
#     "What is the most visited place in Paris? Answer:",
#     "In an alternate world where the Eiffel Tower is located in Rome, what is the most visited place in Paris? Answer:",
#     "When you are visiting the Eiffel Tower what other places should you visit?",
#     "Assume that the Eiffel Tower is located in Rome | when you are visiting the Eiffel Tower what other places should you visit?"
# ]

# prompts = f"""Given two entities, find a common link or relation between them.
# #
# Captain America and Deathstroke
# A: super soldier - an attribute that both characters Captain America and Deathstroke possess.
# #
# Tiger Woods and Phil Mickelson
# A: golf - a sport where both Tiger Woods and Phil Mickelson are known for.
# #
# Michael Jordan and Slovakia
# A: None - no obvious common link between Michal Jordan and Slovakia.
# #
# Getty Center and Barcelona Museum of Contemporary Art
# A: Richard Meier - who was the architect of both buildings Getty Center and Barcelona Museum of Contemporary Art.
# #
# {question}
# A:"""



# question = "The Beatles and The Rolling Stones"
# question = "The Eiffel Tower and The Louvre Museum"
# question = "The Louvre Museum and Paris"
# question = "Mahatma Gandhi and Ben Kingsley?"
# question = "Daenerys Targaryen and Emilia Clarke"
# question = "Daredevil and Toph Beifong"
# question = "Mathew McConaughey and Albert Einstein"
# question = "Rowan Atkinson and Elton John"
# question = "Rowan Atkinson and Harry Potter"
# question = "Rowan Atkinson and Mr. Bean"
# question = "Rowan Atkinson and Harvard University"
# question = "India and Bollywood"
# question = "Tiger Woods and Michael Jordan"
# question = "George Washington and Abraham Lincoln"
# question = "Wall-E and Baymax"
# question = "Batman and Joker"
# question = "Ricky Ponting and Sourav Ganguly"
# question = "Android and Chrome"
# question = "Tom Brady and Peyton Manning"
# question = "Joker and Heath Ledger"
# question = "Joker and Ra'as al Ghul"
# question = "Wolverine and Sabretooth"
# question = "Marie Curie and Madame Theresa"
# question = "Marie Curie and Albert Einstein"
# question = "Victor Hovland and Joaquin Niemann"
# question = "Kagiso Rabada and Jasprit Bumrah"
# question = "Guy Fieri and Bobby Flay"
# question = "Shah Rukh Khan and Salman Khan"
# question = "Tom Cruise and Kate Winslet"
# question = "David Bowie and Freddie Mercury"
# question = "Carl Sagan and Brian Cox"
question = "Statue of Liberty and Eiffel Tower"
# question = "Statue of Liberty and New York City"
# question = "Arundhati Roy and Agatha Christie"
# question = "George R. R. Martin and J. R. R. Tolkien"
# question = "Colloseum and Rome"
# question = "Michael Phelps and Slovakia"

Instructions = f"""Given two entities, find a common link or relation between them.
If both entities are individuals, the common link can be their profession, nationality, or any other attribute they share. Their relation can be if someone is the student/teacher of the other etc.
Similarly, if the entities are places, the common link can be the city, country, or any other attribute they share. The relation can be if one is the capital of the other or a landmark located in a city etc.
If there is no connection just answer "None"."""

# Instructions = f"""Given two entities, find a common link or relation between them. If there is no connection just answer "None"."""

block_separator = "\n#"
question_marker = "\nQ: "
answer_marker = "\nA:"

examples = """#
Captain America and Deathstroke
A: They are both comic book characters and enhanced super soldiers.
#
Q: Tiger Woods and Phil Mickelson
A: They are both professional golfers.
#
Q: Rome and Italy
A: Rome is the capital city of Italy.
#
Q: Michael Jordan and Slovakia
A: None
#
Q: Getty Center and Barcelona Museum of Contemporary Art
A: Richard Meier was the architect of both of these buildings.
"""

prefix = f"""{Instructions}
{examples}
"""

prompt = prepare_probing_input(
    mt=mt,
    entities=question.split(" and "),
    # entities=('Bryce Harper', 'Yadier Molina'),
    prefix=prefix,
    answer_marker=answer_marker,
    question_marker=question_marker,
    block_separator=block_separator,
    is_a_reasoning_model="deepseek" in model_key.lower(),
    # is_a_reasoning_model=True
)

In [10]:
# think_start = mt.tokenizer.encode("<think>", add_special_tokens=False)[0]
# think_end = mt.tokenizer.encode("</think>", add_special_tokens=False)[0]


In [11]:
print(mt.tokenizer.decode(prompt.tokenized["input_ids"][0]))

<｜begin▁of▁sentence｜>Given two entities, find a common link or relation between them.
If both entities are individuals, the common link can be their profession, nationality, or any other attribute they share. Their relation can be if someone is the student/teacher of the other etc.
Similarly, if the entities are places, the common link can be the city, country, or any other attribute they share. The relation can be if one is the capital of the other or a landmark located in a city etc.
If there is no connection just answer "None".
#
Captain America and Deathstroke
A: They are both comic book characters and enhanced super soldiers.
#
Q: Tiger Woods and Phil Mickelson
A: They are both professional golfers.
#
Q: Rome and Italy
A: Rome is the capital city of Italy.
#
Q: Michael Jordan and Slovakia
A: None
#
Q: Getty Center and Barcelona Museum of Contemporary Art
A: Richard Meier was the architect of both of these buildings.
#
Q: Statue of Liberty and Eiffel Tower
A:
Try to keep your think

In [12]:
# print(f"\"{prompt.prompt}\"")
# print(f"{prompt.entity_ranges=} | {prompt.query_range=}")

# print(f"\"{mt.tokenizer.decode(prompt.tokenized['input_ids'][0][prompt.entity_ranges[0][0]:prompt.entity_ranges[0][1]])}\"")
# print(f"\"{mt.tokenizer.decode(prompt.tokenized['input_ids'][0][prompt.entity_ranges[1][0]:prompt.entity_ranges[1][1]])}\"")
# print(f"\"{mt.tokenizer.decode(prompt.tokenized['input_ids'][0][prompt.query_range[0]:prompt.query_range[1]])}\"")

In [13]:
answer = get_lm_generated_answer(
    mt=mt, prompt=prompt, 
    is_a_reasoning_model="deepseek" in model_key.lower()
    # is_a_reasoning_model=True
)
print(f"{answer=}")

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Okay, so I need to figure out the common link or relation between the Statue of Liberty and the Eiffel Tower. Let me start by thinking about what I know about these two iconic structures.

First, the Statue of Liberty is located in New York City, USA. It's a famous landmark, a gift from France, and was built in the late 1800s. It's also known as "Liberty Enlightening the World" and represents the ideals of freedom and democracy.

On the other hand, the Eiffel Tower is in Paris, France. It's one of the most recognizable structures in the world, built in the late 1800s as a gift from France to the United States for the centennial celebration. It's an iron tower and a symbol of Paris and France.

Now, looking for a common link between them. Both were built in the late 19th century, so they share the same time period. They were both gifts from France to the US—France gave the Statue of Liberty, and France offered the Eiffel Tower as a gift, though it was constructed in Paris. They are both

In [14]:
print(answer)

Both are iconic landmarks from the late 19th century, designed by French engineers.


In [86]:
from typing import Optional, Literal
from src.functional import ASK_ORACLE_MODEL

def check_if_answer_is_correct(
    answer: str,
    keywords: list[str],
    oracle_model: Optional[Literal["gpt4o", "claude"]] = "claude",
    entities: tuple[str, str] = None,
) -> bool:
    #! use this only if the answer is not None
    """
    will return true if any keyword is found in the answer.
    if not and the oracle is not None, will ask the oracle model for verification.
    """
    if any([keyword in answer for keyword in keywords]):
        return True
    
    elif oracle_model is not None:
        question = f"""Do you think the following answer is a good connection or relation between the entities {entities[0]} and {entities[1]}?
Your answer should start with "Yes" or "No". If the answer is "No", please provide your reasoning. Otherwise, just say "Yes".
\n\n{answer}"""
        oracle_response = ASK_ORACLE_MODEL[oracle_model](prompt=question)
        logger.debug(f"{oracle_response=}")
        return oracle_response.startswith("Yes")
    
    return False


keywords = [
        "scientist",
        "research",
        "inventor",
        "discovery",
        "experiment",
        "laboratory",
        "science"
    ]
oracle_model = "claude"

check_if_answer_is_correct(
    answer=answer,
    keywords=keywords,
    oracle_model=oracle_model,
    entities=prompt.entities,
)

2025-01-31 13:42:34 httpx DEBUG    load_ssl_context verify=True cert=None trust_env=True http2=False
2025-01-31 13:42:34 httpx DEBUG    load_verify_locations cafile='/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/certifi/cacert.pem'


2025-01-31 13:42:34 anthropic._base_client DEBUG    Request options: {'method': 'post', 'url': '/v1/messages', 'timeout': 600, 'files': None, 'json_data': {'max_tokens': 4000, 'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Do you think the following answer is a good connection or relation between the entities Statue of Liberty and Eiffel Tower?\nYour answer should start with "Yes" or "No". If the answer is "No", please provide your reasoning. Otherwise, just say "Yes".\n\n\n\\text{Both are iconic landmarks from the late 19th century, designed by French engineers.'}]}], 'model': 'claude-3-5-sonnet-20241022', 'system': 'You are a helpful assistant.', 'temperature': 0}}
2025-01-31 13:42:34 anthropic._base_client DEBUG    Sending HTTP Request: POST https://api.anthropic.com/v1/messages
2025-01-31 13:42:34 httpcore.connection DEBUG    connect_tcp.started host='api.anthropic.com' port=443 local_address=None timeout=600 socket_options=None
2025-01-31 13:42:34 httpcore.con

True

In [170]:
from src.functional import get_hs
import itertools

layer_names = mt.layer_names
token_indices = set(
    set(range(*prompt.query_range))
    .union(set(range(*prompt.entity_ranges[0])))
    .union(set(range(*prompt.entity_ranges[1])))
)

token_indices = list(range(min(token_indices), max(token_indices)+1))

locations = list(itertools.product(layer_names, token_indices))

latents = get_hs(
    mt=mt,
    input=TokenizerOutput(prompt.tokenized),
    locations=locations,
    return_dict=True,
)

In [229]:
from src.utils.typing import ArrayLike
from src.functional import detensorize

def map_latent_keys_to_str(dct):
    cpy = {}
    for key, value in dct.items():
        cpy["_<>_".join([str(k) for k in key])] = value
    return cpy

@dataclass
class ProbingLatents(DataClassJsonMixin):
    prompt: ProbingPrompt
    latents: dict[tuple[str, int] | str, ArrayLike]

    def get_dict(self):
        dct = map_latent_keys_to_str(self.latents)
        dct.update(self.prompt.to_dict())
        return dct
    
    @staticmethod
    def from_npz(npz_file):
        probing_keys =  ['prompt', 'entities', 'model_key', 'tokenized', 'entity_ranges', 'query_range']
        prompt = ProbingPrompt(
            prompt = npz_file["prompt"].item(),
            entities = npz_file["entities"].tolist(),

            model_key = npz_file["model_key"].item(),
            tokenized = npz_file["tokenized"].item(),

            entity_ranges=npz_file["entity_ranges"].tolist(),
            query_range=npz_file["query_range"].tolist(),
        )

        latents = {}
        for key in npz_file.files:
            if key not in probing_keys and key != "allow_pickle":
                location = key.split("_<>_")
                location[1] = int(location[1])
                latents[tuple(location)] = torch.Tensor(npz_file[key])

        return ProbingLatents(
            prompt=prompt,
            latents=latents,
        ) 
            

probing_latents = ProbingLatents(
    prompt=prompt,
    latents=latents,
)

In [230]:
dten = detensorize(probing_latents, to_numpy=False)
dten.latents = map_latent_keys_to_str(dten.latents)

with open("test.json", "w") as f:
    f.write(dten.to_json())

In [216]:
# import json
# with open("test.json", "r") as f:
#     data = json.load(f)
# djson = ProbingLatents.from_dict(data)

In [270]:
import numpy as np

dnpz = detensorize(probing_latents, to_numpy=True)
# np.savez_compressed("test.npz", **dnpz.get_dict(), allow_pickle=True)
np.savez_compressed("test.npz", **dnpz.get_dict(), allow_pickle=True)

In [47]:
import numpy as np

dnpz = np.load("/home/local_arnab/Codes/Projects/retrieval/results/probing_test/Llama-3.2-3B/baseball/3edb860e7aa4b56350161859a03af888.npz", allow_pickle=True)

In [48]:
dnpz_loaded = ProbingLatents.from_npz(dnpz)

In [49]:
dnpz_loaded.prompt.entities

['Walter Johnson', 'Prince Fielder']

In [51]:
dnpz_loaded.lm_answer